In [54]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the dataset from a CSV file
file_path = "Data Gathering Survey (Responses).csv"
df = pd.read_csv(file_path)

print("Which university are you in?" in df.columns)
print(df['Which university are you in?'].head())

missing_cols = [col for col in categorical_features if col not in df.columns]
if missing_cols:
    print(f"Warning: The following columns are missing: {missing_cols}")


# Encode categorical variables
categorical_features = [
    "What is your gender?", 
    "What is your Nationality?", 
    "Are you a domestic or International Student?", 
    "What is your highest academic qualification before joining university?",
    "How would you describe your personality?", 
    "Are you involved in any leadership roles (e.g., student council, club president)?",
    "What type of extracurricular activities do you prioritize?",
    "What sources did you rely on to fund your university education?",
    "What was your GPA/Grade of your highest qualification? (E.g. 3.6/4.0 GPA, 70/90 Rank Points)"
]

numerical_features = [
    "How many hours per week do you dedicate to extracurricular activities?",
    "How significant was the availability of financial aid or scholarships in your university choice?"
]

# Prepare X and y
X = df[categorical_features + numerical_features]  # Select relevant columns
y = df['Which university are you in?']  # Assuming this is your target variable

# Encode categorical features and scale numerical ones
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit and transform the data using the preprocessor
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Encode target variable
y_encoder = LabelEncoder()
y_train = y_encoder.fit_transform(y_train)
y_test = y_encoder.transform(y_test)

# Define neural network model
model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(len(np.unique(y_train)), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')

True
0    National University of Singapore (NUS)
1    National University of Singapore (NUS)
2    Nanyang Technological University (NTU)
3     Singapore Management University (SMU)
4    Nanyang Technological University (NTU)
Name: Which university are you in?, dtype: object
Epoch 1/50


C:\Users\jeowc\anaconda3\envs\DNN\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_3/dense_9/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_3/dense_9/embedding_lookup_sparse/Reshape:0", shape=(None, 64), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_3/dense_9/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


2/2 [==============================] - 1s 224ms/step - loss: 1.6038 - accuracy: 0.1429 - val_loss: 1.7287 - val_accuracy: 0.0000e+00
Epoch 2/50
2/2 [==============================] - 0s 18ms/step - loss: 1.5314 - accuracy: 0.1786 - val_loss: 1.6719 - val_accuracy: 0.0000e+00
Epoch 3/50
2/2 [==============================] - 0s 20ms/step - loss: 1.4731 - accuracy: 0.2500 - val_loss: 1.6144 - val_accuracy: 0.2500
Epoch 4/50
2/2 [==============================] - 0s 18ms/step - loss: 1.4201 - accuracy: 0.3929 - val_loss: 1.5627 - val_accuracy: 0.2500
Epoch 5/50
2/2 [==============================] - 0s 20ms/step - loss: 1.3711 - accuracy: 0.5714 - val_loss: 1.5132 - val_accuracy: 0.2500
Epoch 6/50
2/2 [==============================] - 0s 17ms/step - loss: 1.3232 - accuracy: 0.6786 - val_loss: 1.4676 - val_accuracy: 0.2500
Epoch 7/50
2/2 [==============================] - 0s 17ms/step - loss: 1.2815 - accuracy: 0.7143 - val_loss: 1.4238 - val_accuracy: 0.5000
Epoch 8/50
2/2 [=============

In [61]:
# Function to predict university from user input
def predict_university(user_input):
    # Create a DataFrame for the user input with the correct columns order
    user_df = pd.DataFrame([user_input])
    
    # Preprocess the data using the same preprocessor pipeline
    user_processed = preprocessor.transform(user_df)
    
    # Get prediction probabilities
    prediction_probs = model.predict(user_processed)
    
    # Get indices of top 3 predicted universities (highest probabilities)
    top_3_indices = prediction_probs[0].argsort()[-3:][::-1]
    
    # Get the top 3 predicted universities
    top_3_universities = y_encoder.inverse_transform(top_3_indices)
    
    # Get the corresponding probabilities for the top 3 predictions
    top_3_scores = prediction_probs[0][top_3_indices]
    
    # Return top 3 predicted universities along with their scores
    return list(zip(top_3_universities, top_3_scores))

# Example user input (modify accordingly)
user_input_example = {
    "What is your gender?": "Male",
    "What is your Nationality?": "Singaporean",
    "Are you a domestic or International Student?": "Domestic",
    "What is your highest academic qualification before joining university?": "A-Level",
    "What was your GPA/Grade of your highest qualification? (E.g. 3.6/4.0 GPA, 70/90 Rank Points)": "3.6/4.0",
    "How many hours per week do you dedicate to extracurricular activities?": 0,
    "How significant was the availability of financial aid or scholarships in your university choice?": 10,
    "How would you describe your personality?": "Extrovert",
    "Are you involved in any leadership roles (e.g., student council, club president)?": "Yes",
    "What type of extracurricular activities do you prioritize?": "Sports",
    "What sources did you rely on to fund your university education?": "Scholarships"
}

# Get prediction
predicted_universities = predict_university(user_input_example)
print("Top 3 predicted universities:")
for uni, score in predicted_universities:
    print(f"{uni}: {score:.4f}")


1/1 [==============================] - 0s 12ms/step
Top 3 predicted universities:
Singapore Management University (SMU): 0.4782
Singapore Institute of Technology (SIT): 0.3557
Nanyang Technological University (NTU): 0.0759
